In [1]:
import numpy as np
import pandas as pd
import pathlib
import os
import glob
from time import sleep,time
from datetime import datetime
from IPython.core import display as ICD

import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 100)

In [2]:
class tictactoe:
    def __init__(self,
                 path=str(pathlib.Path().resolve())+'/Database/'):
        self.path=path
        self.matrix="----------------\n|  1 | 2  | 3  |\n----------------\n| 4  | 5  | 6  |\n----------------\n| 7  | 8  | 9  |\n----------------\n"
        self.turn_map={'X':1,'O':-1}
        self.win_combinations=[[1,2,3],[4,5,6],[7,8,9],
                               [1,4,7],[2,5,8],[3,6,9],
                               [1,5,9],[3,5,7]]
        self.cell_index={'1':20,'2':24,'3':29,
                         '4':53,'5':58,'6':63,
                         '7':87,'8':92,'9':97}
        self.reset_data()
        self.reset_game()
        
    def replace_cell_display(self,index):

        i=str(index)
        self.play_matrix=self.play_matrix[:self.cell_index[i]] + self.turn + self.play_matrix[self.cell_index[i]+1:]

    def replace_cell_array(self,index):
        self.arr[index-1]=self.turn_map[self.turn]
    def check_win_criteria(self):
        for i in self.win_combinations:
            if all(self.arr[index-1] == self.turn_map[self.turn] for index in i):
                return 1
        return 0
    def reset_game(self):
        self.play_matrix = "----------------\n|    |    |    |\n----------------\n|    |    |    |\n----------------\n|    |    |    |\n----------------\n"
        self.entry_count=0
        self.turn='X'
        self.arr=np.array([0,0,0,0,0,0,0,0,0])
        self.switch_turn=0
        self.break_=False
        
        
    def reset_data(self):
        self.columns=['player','win']+[str(i)+"_played" for i in range(1,10)]+\
                     [str(i)+"_threat" for i in range(1,10)]+\
                     [str(i)+"_win" for i in range(1,10)]+['next_move']
        self.data=pd.DataFrame(columns=self.columns)
        
        print("...Reset data...")
    
    def check_status(self):
        map_check = {'X':[2,-2],
                     'O':[-2,2]}
        for c in self.win_combinations:
            l=[self.arr[i-1] for i in c]
            if sum(l)==map_check[self.turn][0]:
                self.data.iloc[self.data_index,19+c[l.index(0)]]=1 # cell for potential win
            if sum(l)==map_check[self.turn][1]:
                self.data.iloc[self.data_index,10+c[l.index(0)]]=1 # cell for potential loss
    
    def new_game_request(self):
        n = input("New game? (y/n): ")
        if n=='y':
            print("--------------------------------------------------")
            print("NEW GAME : ")
            self.reset_game()
            self.data_index+=1
        else:
            print("Thank you for playing......")
            sleep(3)
            self.saved=self.check_game_data()
            self.break_=True
    def save_data(self):
        self.data.to_csv(self.path+f'Game_{str(datetime.now())}.csv')
    def play(self):
        self.reset_game()
        self.reset_data()
        self.data_index=0
        while(1):
            print("--------------------------------------------------")
            self.data.loc[self.data.shape[0]] = [0]*len(self.data.columns)
            self.switch_turn=1
            print(self.arr)
            print(self.matrix)
            print(self.play_matrix)
            if self.entry_count==9:
                print("Draw...")
                self.new_game_request()
                if self.break_:
                    self.data['next_move']=self.data['next_move'].shift(-1)
                    self.save_data()
                    break
                self.data.loc[self.data.shape[0]] = [0]*len(self.data.columns)
                self.switch_turn=1
                print(self.arr)
                print(self.matrix)
                print(self.play_matrix)
            
            inp=int(input(f"Enter cell index to place {self.turn} : "))
            c = str(inp)
            
            # enter 0 to stop the game
            if inp==0:
                self.new_game_request()
                if self.break_:
                    self.data['next_move']=self.data['next_move'].shift(-1)
                    self.save_data()
                    break
                
            # check for invalid entries
            if inp>9:
                self.switch_turn=0
                print("try a valid cell")
            elif self.play_matrix[self.cell_index[c]] in ['X','O']:
                self.switch_turn=0
                print("try another cell")

            # operations after a valid entry
            else:
                
                self.entry_count+=1
                self.replace_cell_array(inp)
                self.replace_cell_display(c)
                self.data.iloc[self.data_index,-1]   = inp
                self.data.iloc[self.data_index,0]    = self.turn
                self.data.iloc[self.data_index,2:11] = self.arr
                self.check_status()

                if self.check_win_criteria()==1:
                    self.data.iloc[self.data_index,1]=1
                    print(self.play_matrix)
                    print(f"Player {self.turn} wins the game.")
                    self.new_game_request()
                    if self.break_:
                        self.data['next_move']=self.data['next_move'].shift(-1)
                        self.save_data()
                        break


            # operations before next turn
            if self.switch_turn==1:
                self.data_index+=1
                if self.turn=='X':
                    self.turn='O'
                else:
                    self.turn='X'
                    
    
    def check_game_data(self):
        return self.data
    
    def combine_existing_data(self):
        csv_files = glob.glob(os.path.join(self.path, "*.csv"))
        comb_df = pd.DataFrame(columns=self.data.columns)
        for i in csv_files:
            temp = pd.read_csv(i)
            comb_df = pd.concat([comb_df,temp],ignore_index=True)
        return comb_df
        
        
        
        
        

In [3]:
ttt=tictactoe()
ttt.play()

...Reset data...
...Reset data...
--------------------------------------------------
[0 0 0 0 0 0 0 0 0]
----------------
|  1 | 2  | 3  |
----------------
| 4  | 5  | 6  |
----------------
| 7  | 8  | 9  |
----------------

----------------
|    |    |    |
----------------
|    |    |    |
----------------
|    |    |    |
----------------

Enter cell index to place X : 6
--------------------------------------------------
[0 0 0 0 0 1 0 0 0]
----------------
|  1 | 2  | 3  |
----------------
| 4  | 5  | 6  |
----------------
| 7  | 8  | 9  |
----------------

----------------
|    |    |    |
----------------
|    |    | X  |
----------------
|    |    |    |
----------------

Enter cell index to place O : 2
--------------------------------------------------
[ 0 -1  0  0  0  1  0  0  0]
----------------
|  1 | 2  | 3  |
----------------
| 4  | 5  | 6  |
----------------
| 7  | 8  | 9  |
----------------

----------------
|    | O  |    |
----------------
|    |    | X  |
------------

Enter cell index to place X : 7
--------------------------------------------------
[ 1 -1  1  0  0  0  1  0 -1]
----------------
|  1 | 2  | 3  |
----------------
| 4  | 5  | 6  |
----------------
| 7  | 8  | 9  |
----------------

----------------
|  X | O  | X  |
----------------
|    |    |    |
----------------
| X  |    | O  |
----------------

Enter cell index to place O : 4
--------------------------------------------------
[ 1 -1  1 -1  0  0  1  0 -1]
----------------
|  1 | 2  | 3  |
----------------
| 4  | 5  | 6  |
----------------
| 7  | 8  | 9  |
----------------

----------------
|  X | O  | X  |
----------------
| O  |    |    |
----------------
| X  |    | O  |
----------------

Enter cell index to place X : 5
----------------
|  X | O  | X  |
----------------
| O  | X  |    |
----------------
| X  |    | O  |
----------------

Player X wins the game.
New game? (y/n): n
Thank you for playing......


In [6]:
ttt.combine_existing_data().to_csv('batch_1_data.csv')

In [13]:
csv_files

['/Users/eashwar/Documents/Projects/TicTacToe/Database/Game_2021-11-18 16:10:18.028838.csv',
 '/Users/eashwar/Documents/Projects/TicTacToe/Database/Game_2021-11-19 11:13:59.351219.csv',
 '/Users/eashwar/Documents/Projects/TicTacToe/Database/Game_2021-11-18 16:26:34.571959.csv',
 '/Users/eashwar/Documents/Projects/TicTacToe/Database/Game_2021-11-18 15:08:02.856201.csv',
 '/Users/eashwar/Documents/Projects/TicTacToe/Database/Game_2021-11-18 16:23:24.949517.csv',
 '/Users/eashwar/Documents/Projects/TicTacToe/Database/Game_2021-11-19 11:12:45.592468.csv']

In [7]:
ttt.data

,player,win,1_played,2_played,3_played,4_played,5_played,6_played,7_played,8_played,9_played,1_threat,2_threat,3_threat,4_threat,5_threat,6_threat,7_threat,8_threat,9_threat,1_win,2_win,3_win,4_win,5_win,6_win,7_win,8_win,9_win,next_move
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN
